In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

In [3]:
fall=r"E:\Fall Detection Dataset\fall_cam1\datasets"
nonfall=r"E:\Fall Detection Dataset\nonfall_cam0"

video_data=[]
labels=[]

In [4]:
for video_folder in sorted(os.listdir(fall)):
    video_path = os.path.join(fall, video_folder)
    
    if not os.path.isdir(video_path):
        continue  # Skip if it's not a folder
    
    frames = []
    
    # Iterate through frames (images) in sorted order
    for frame in sorted(os.listdir(video_path)):
        frame_path = os.path.join(video_path, frame)
        
        if frame_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(frame_path)  # Read image
            #img = cv2.resize(img, frame_size)  # Resize (optional)
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            frames.append(img)
    
    if frames:  # If frames were found
        video_array = np.array(frames)  # Convert list to NumPy array
        video_data.append(video_array)  # Append to dataset
        labels.append(video_folder)  # Store the folder name as label



In [5]:
# Convert to NumPy arrays
labels=np.array(labels)
#video_data = np.array(video_data, dtype=object)  # Each video has a different number of frames
print(f"Total videos loaded: {len(video_data)}")
print(f"Shape of first video: {video_data[0].shape}")

Total videos loaded: 30
Shape of first video: (160, 480, 640, 3)


In [6]:
min_frames=min(video.shape[0] for video in video_data)
min_frames

55

In [7]:
len(video_data)

30

In [8]:
video_data[0] # this is first video

array([[[[175, 168, 181],
         [174, 167, 180],
         [173, 166, 179],
         ...,
         [254, 254, 254],
         [254, 254, 254],
         [254, 254, 254]],

        [[178, 171, 184],
         [177, 170, 183],
         [175, 168, 181],
         ...,
         [254, 254, 254],
         [254, 254, 254],
         [254, 254, 254]],

        [[176, 169, 182],
         [174, 167, 180],
         [173, 166, 179],
         ...,
         [254, 254, 254],
         [254, 254, 254],
         [254, 254, 254]],

        ...,

        [[ 40,  37,  33],
         [ 41,  38,  34],
         [ 41,  38,  34],
         ...,
         [ 89,  89,  75],
         [ 94,  94,  80],
         [ 94,  94,  80]],

        [[ 42,  39,  35],
         [ 43,  40,  36],
         [ 43,  40,  36],
         ...,
         [ 89,  90,  74],
         [ 91,  92,  76],
         [ 89,  90,  74]],

        [[ 44,  41,  37],
         [ 44,  41,  37],
         [ 44,  41,  37],
         ...,
         [ 88,  89,  73],
        

In [9]:
video_data[0].shape

(160, 480, 640, 3)

In [10]:
sliced_videos = []
for video in video_data:
    # Calculate the step size
    num_frames = video.shape[0]
    step = num_frames / min_frames  # Compute interval step

    # Select frames at regular intervals
    selected_indices = np.round(np.arange(0, num_frames, step)).astype(int)[:min_frames] # extracts indices
    sliced_video = video[selected_indices] #slices the video
    sliced_videos.append(sliced_video) # adds sliced video to list
        

In [11]:
sliced_videos[29].shape

(55, 480, 640, 3)

In [12]:
sliced_videos = np.array(sliced_videos)

In [13]:
sliced_videos.shape

(30, 55, 480, 640, 3)

In [14]:
# Load MoveNet model
interpreter = tf.lite.Interpreter(model_path=r"E:\jupyter notebook\3.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to run MoveNet on an image
def run_movenet(image):
    image_resized = cv2.resize(image, (192, 192))  # Resize to model input
    image_resized = np.expand_dims(image_resized, axis=0).astype(np.float32) / 255.0  # Normalize

    interpreter.set_tensor(input_details[0]['index'], image_resized)
    interpreter.invoke()
    
    keypoints = interpreter.get_tensor(output_details[0]['index'])[0][0][:, :2]  # (17,2)
    return keypoints

# Process video frames
def process_videos(video_frames):
    num_videos, num_frames, _, _, _ = video_frames.shape
    keypoints_data = np.zeros((num_videos, 17 * 2 * num_frames))  # Shape (30, 1870)

    for v in range(num_videos):
        video_keypoints = []
        for f in range(num_frames):
            keypoints = run_movenet(video_frames[v, f])  # (17, 2)
            video_keypoints.append(keypoints.flatten())  # Flatten to (34,)

        keypoints_data[v] = np.concatenate(video_keypoints)  # Flattened shape (1870,)

    return keypoints_data


In [15]:
pro_video=process_videos(sliced_videos)

In [16]:
pro_video.shape

(30, 1870)

In [18]:
# Combine labels and data
data_with_labels = np.column_stack((pro_video,labels))
data_with_labels

array([['0.7133227586746216', '0.16475749015808105',
        '0.7013038396835327', ..., '0.6213837265968323',
        '0.3814561069011688', 'fall-01-cam0-rgb'],
       ['0.7133110761642456', '0.1654997020959854', '0.7011114954948425',
        ..., '0.6200836300849915', '0.38198322057724',
        'fall-02-cam0-rgb'],
       ['0.7134003639221191', '0.16458655893802643',
        '0.7013654112815857', ..., '0.6211385726928711',
        '0.3819645345211029', 'fall-03-cam0-rgb'],
       ...,
       ['0.7168795466423035', '0.169572114944458', '0.6867665648460388',
        ..., '0.6197090148925781', '0.383663535118103',
        'fall-28-cam0-rgb'],
       ['0.7168178558349609', '0.1675603985786438', '0.6864893436431885',
        ..., '0.6197540163993835', '0.38353031873703003',
        'fall-29-cam0-rgb'],
       ['0.7164229154586792', '0.16956260800361633',
        '0.6862401962280273', ..., '0.619764506816864',
        '0.38358208537101746', 'fall-30-cam0-rgb']], dtype='<U32')

In [19]:
np.savetxt("output3.csv", data_with_labels, delimiter=",", fmt="%s")

print("CSV file saved successfully.")

CSV file saved successfully.


In [20]:
import pandas as pd

In [21]:
dataset = pd.read_csv("output3.csv",header=None)

In [22]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870
0,0.713323,0.164757,0.701304,0.186173,0.603872,0.501781,0.552190,0.355947,0.489820,0.544001,...,0.414629,0.590906,0.568317,0.562365,0.367789,0.641429,0.971560,0.621384,0.381456,fall-01-cam0-rgb
1,0.713311,0.165500,0.701111,0.186969,0.647033,0.010800,0.552291,0.357098,0.469514,0.542455,...,0.416321,0.590961,0.568286,0.561157,0.368119,0.641389,0.971350,0.620084,0.381983,fall-02-cam0-rgb
2,0.713400,0.164587,0.701365,0.185978,0.603724,0.501218,0.552267,0.355447,0.468900,0.542743,...,0.414835,0.590553,0.569103,0.561606,0.367483,0.641407,0.971463,0.621139,0.381965,fall-03-cam0-rgb
3,0.713343,0.165221,0.701165,0.186709,0.647200,0.010567,0.552450,0.356670,0.449069,0.542097,...,0.416314,0.591111,0.567495,0.561815,0.368866,0.641391,0.971454,0.620209,0.382163,fall-04-cam0-rgb
4,0.713286,0.164676,0.701260,0.186067,0.603808,0.501849,0.552240,0.355974,0.489868,0.544099,...,0.413926,0.590713,0.568685,0.562874,0.367234,0.641285,0.971314,0.607210,0.384894,fall-05-cam0-rgb


In [38]:
np.unique(data_with_labels[:,-1]).shape

(30,)